In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import os
import sys
sys.path.insert(0, os.path.abspath('..'))
from psycopg2 import connect
from dotenv import load_dotenv
from labevents import LabEventDistributions, get_valid_values_from_labevents_for_itemid
import statistics
from db import PostgresDB

In [4]:
def get_mean_std_for_item_id(self, item_id: int):
    if item_id in self._mean_std_cache:
        return None, None
    values = get_valid_values_from_labevents_for_itemid(
        labevents=self.labevents, item_id=item_id
    )
    units = [o.valueuom for o in self.labevents if o.item_id == item_id]
    if not len(set(units)) == 1:
        print(f"Multiple units found for item_id: {item_id}")
    if len(values) > 1:
        mean = statistics.mean(values)
        std = max(statistics.stdev(values), 0.0001)
        self._mean_std_cache[item_id] = mean, std
        return mean, std
    else:
        return None, None

In [17]:
db = PostgresDB(
    db_name=os.getenv("DB_NAME"),
    host=os.getenv("DB_HOST"),
    port=os.getenv("BD_PORT"),
    user=os.getenv("DB_USER"),
)


Connected to database


In [21]:
db.get_mean_std_for_itemid(50868)

(14.196377976228614, 3.4285864990839574)